In [822]:
import pandas as pd
import numpy as np
import json

Entropy for set "S":
$$ E(S) = -\sum_{i=1}^{n} P_i \log_2 P_i $$

Here, 

$S$ is a set of data.

$P_i$ is the probability of occuring of $target_i$ within the set.

In [823]:
def entropy(s):
    map = {}  # dictionary to hold target values and their counts
    
    if isinstance(s, pd.Series):
        datasetLength = len(s)
        for i in range(datasetLength):
            val = s.iloc[i]
            map[val] = map.get(val, 0) + 1
    elif isinstance(s, dict):
        datasetLength = sum(s.values())  # total count
        map = s
    else:
        raise TypeError("Input must be a pandas Series or a dictionary")

    ent = 0
    for val in map:
        p = map[val] / datasetLength
        ent += -p * np.log2(p)
    
    return ent

Gain function

$$Gain(S, A) = E(S)-\sum_{v \varepsilon Values(A)} \left(\frac{|S_v|}{|S|}E(S_v) \right)$$

Here,

$S$ is a set of data

$A$ is a feature within the set

$S_v$ is the subset for the value of the feature

In [824]:
def getGain(s, a):
    datasetLength = len(s)
    attributeMap = {}  # dictionary to hold the number of occurrences of each attribute value
    
    for i in range(datasetLength):
        attr_val = a.iloc[i]
        target_val = s.iloc[i]
        
        if attr_val in attributeMap:
            if target_val in attributeMap[attr_val]:
                attributeMap[attr_val][target_val] += 1
            else:
                attributeMap[attr_val][target_val] = 1
        else:
            attributeMap[attr_val] = {target_val: 1}
    
    avgEntropy = 0  # total entropy of the attribute values
    attributeEntropies = {}  # dictionary to hold the entropy of each attribute value

    for attribute in attributeMap:
        totalValues = sum(attributeMap[attribute].values())
        attrEntropy = entropy(attributeMap[attribute])
        attributeEntropies[attribute] = attrEntropy
        avgEntropy += (totalValues / datasetLength) * attrEntropy
    
    gain = entropy(s) - avgEntropy
    attributeEntropies = dict(sorted(attributeEntropies.items(), key=lambda item: item[1]))
    
    return gain, attributeEntropies


In [825]:
df = pd.read_excel('binaryData.xlsx')
target = df.columns[-1]  # last column is the target variable
# use integer code for target variable
df[target] = df[target].astype('category').cat.codes # assigns 0 for 'no' and 1 for 'yes'
df

,Vehicle,Speed,Contact direction,body part hit,life
0,Car,high,direct,limb,1
1,Car,high,side,chest,1
2,Car,low,angle,head,1
3,Car,low,angle,chest,0
4,Car,low,direct,limb,0
5,Truck,high,direct,head,1
6,Truck,low,angle,chest,1
7,Truck,high,side,limb,1
8,Bike,high,direct,head,1
9,Bike,low,side,chest,0


In [826]:
def selectParentNode(s, target):
    maxGain = -1
    parentNode = None
    attributeEntropies = {}
    
    for col in s.columns:
        if col != target:
            gain, attrEntropies = getGain(s[target], s[col])
            if gain > maxGain:
                maxGain = gain
                parentNode = col
                attributeEntropies = attrEntropies
            
    return parentNode, attributeEntropies

In [827]:
def createTree(dataset, target, tree={}, originNode = None):
    parentNode, attributeEntropies = selectParentNode(dataset, target)
    tree[parentNode] = attributeEntropies
    tree[parentNode]['from'] = originNode
    # attributes with entropy == 0 will be replaced by their result and non-zero entrop attributes with be set to None
    for attribute in attributeEntropies:
        if attribute == 'from':
            continue
        if tree[parentNode][attribute] == 0:
            tree[parentNode][attribute] = dataset[dataset[parentNode] == attribute].iloc[0][target] # leaf node's entropy is replaced by it's decision
        else:
            if tree[parentNode][attribute] != None:
                tree[parentNode][attribute] = None
                subset = dataset[dataset[parentNode] == attribute]
                if len(subset) == 0:
                    return tree
                else:
                    createTree(subset, target, tree, originNode = f'{parentNode},{attribute}')
    return tree
treeDict = createTree(df, target)
treeDict

{'Vehicle': {'Truck': 1, 'Car': None, 'Bike': None, 'from': None},
 'Speed': {'high': 1, 'low': None, 'from': 'Vehicle,Car'},
 'body part hit': {'head': 1, 'chest': 0, 'limb': 0, 'from': 'Speed,low'},
 'Contact direction': {'direct': 1,
  'side': 0,
  'angle': 0,
  'from': 'Vehicle,Bike'}}

In [828]:
# arranges the tree data
def arrangeTreeDict(treeDict):
    def getHiararcy(treeDict):
        hiararcy = {} # stores the hiararcy of data (key = data and value = hiararcy the more the hiararcy, the depper the data is located)\
        while len(hiararcy) != len(treeDict):
            for feature in treeDict:
                if not feature in hiararcy:
                    if treeDict[feature]['from'] == None: # if it's a root node, add hiaracy score to 0
                        hiararcy[feature] = 0
                    else:
                        parent = treeDict[feature]['from'].split(',')[0]
                        if parent in hiararcy:
                            hiararcy[feature] = hiararcy[parent]+1 # the hiararcy of a feature will be +1 than it's paren't hiararcy
        hiararcy = sorted(hiararcy, key=lambda k: hiararcy[k], reverse=True) # sort in descending order
        return hiararcy # index of teh hiararcy array directly represents it's hiararcy. Lower values mean deeper.
    arranged = {}
    hiararcy = getHiararcy(treeDict)
    for index, feature in enumerate(hiararcy):
        if index == len(hiararcy)-1: # skip the last feature (root node)
            del arranged[feature]['from']
            continue
        parent = treeDict[feature]['from'].split(',')
        parentFeature = parent[0]
        parentAttr = parent[1]
        arranged[parentFeature] = treeDict[parentFeature]
        if not feature in arranged:
            del treeDict[feature]['from']
            arranged[parentFeature][parentAttr] = {feature: treeDict[feature]}
        else:
            del arranged[feature]['from']
            arranged[parentFeature][parentAttr] = {feature: arranged[feature]}
            del arranged[feature]
    return arranged
    
treeDict = arrangeTreeDict(treeDict)
treeDict

{'Vehicle': {'Truck': 1,
  'Car': {'Speed': {'high': 1,
    'low': {'body part hit': {'head': 1, 'chest': 0, 'limb': 0}}}},
  'Bike': {'Contact direction': {'direct': 1, 'side': 0, 'angle': 0}}}}

In [829]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.generic):
            return obj.item()
        return super().default(obj)

jsonTree = json.dumps(treeDict, indent=4, cls=NumpyEncoder)
print(jsonTree)

{
    "Vehicle": {
        "Truck": 1,
        "Car": {
            "Speed": {
                "high": 1,
                "low": {
                    "body part hit": {
                        "head": 1,
                        "chest": 0,
                        "limb": 0
                    }
                }
            }
        },
        "Bike": {
            "Contact direction": {
                "direct": 1,
                "side": 0,
                "angle": 0
            }
        }
    }
}


In [830]:
# Write dictionary to a txt file
with open('tree.json', 'w') as file:
    file.write(str(jsonTree))

Run visualizeTree.html to see the tree